In [7]:
from bs4 import BeautifulSoup
from lxml.html import parse
import urllib.request, json
from urllib.request import urlopen
from urllib.error import URLError
from lxml import objectify
from urllib import request
import urllib3
import pandas as pd
import numpy as np
import pdfkit
from column_standard import variable_standardize
import datetime
import os
from urllib.parse import urlparse

In [1]:
config = pdfkit.configuration(wkhtmltopdf='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe')
pdfkit.from_url('http://www.pbaco.org/Physicians', r'\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM708_Dellva\Working\Compliance\Compliance_Data\Ad_Hoc_Requests\out.pdf', configuration = config)


NameError: name 'pdfkit' is not defined

In [19]:
class WebRoot():
    def __init__(self, root, aco_id = "default"):
        self.aco_id = aco_id
        self.root = root
        self.site_links = [self.root]
        self.tested_links = []
        self.tested_suffixes = []
        
    def __str__(self):
        return self.root
    
    def __repr__(self):
        return self.root
    
    def get_testroot(self):
        try:
            self.test_root = self.root.replace("www.","")
            self.test_root = self.test_root.replace("http://","")
            self.test_root = self.test_root.replace("https://","")
            
        except AttributeError:
            print("Error in test_root for {}.\n Root is {}".format(self.aco_id, self.root))
            self.test_root = ""
                
    def extract(self):
        """This is the primary function that should get called to extract the links for the ACO"""
        n = 0
        
        last_iter_links = []
        
        while self.sort_list(self.site_links) != self.sort_list(self.tested_links):
            
            if self.sort_list(self.site_links) == self.sort_list(last_iter_links):
                break
                
            print("Iteration", n)
            print("Site Links are:")
            print([x for x in self.site_links])
            
            print("Tested Links are:")
            print([x for x in self.tested_links])
            
            self.cycle_links()
            
            last_iter_links = self.site_links
            
#             n += 1
            
#             if n == 5:
#                 break

    def cycle_links(self):
        """Cycles through all links in site_links and calls extract_all_links if the url has not already been tested"""
        
        self.site_links = [x for x in self.site_links if x is not None]
        
        for link in self.site_links:
            if link not in self.tested_links and self.drop_lead_dirnav(link) not in self.tested_suffixes:
                print("Extracting links for {}".format(link))
                self.extract_url_links(link)
            
    def extract_url_links(self, url):
        """Extracts all href from the passed url and drops them in self.site_links
           Then, puts the tested url in self.tested_links"""
        
        #cleaning on the url
        url = self.test_http(url)
        url = self.drop_lead_dirnav(url)
        
        #extract the html from the url
        read_text = self.test_url(url)
        
        if read_text is not None:
            #add that link to the links that you've tested
            self.tested_links.append(url)
        else:
            #this will break the loop if the url isn't valid
            return None
                            
        #create a BeautifulSoup instance of the html for easier testing
        soup = BeautifulSoup(read_text,"lxml")
        
        #get all href links from this url
        url_href_links = self.find_href(url, soup)
        
        #iterate through all of those links and append them to site_links if they aren't already there
        [self.site_links.append(x) for x in url_href_links if x not in url_href_links]
        
    def find_href(self, url_root, soup):
        """Accepts an instance of BeautifulSoup and returns all the href contained within"""
#         self.get_testroot()
        
        return_list = []
        
        #BeautifulSoup function find_all will find all <a> tags and we'll iterate through them to extract their hrefs
        for link in soup.find_all("a"):
            print(link)
            #set the initial error flag to 0
            urlerror = 0

            #get the href from the <a> element
            test_link = link.get('href')

            if test_link is not None:
                p_url = urlparse(test_link, scheme="http", allow_fragments = False)
                print("parsed url is: {}".format(p_url))
                clean_url = p_url.geturl()
                print("cleaned url is: {}".format(clean_url))
                if p_url[0] == 'http':
                    try:
                        clean_url = urllib.request.urlopen(clean_url).geturl()
                    except:
                        urlerror = urllib.request.urlopen(clean_url).getcode()
                        print("Tested URL is {}. Response code for this URL is {}".format(clean_url, url_error))
                        continue
                else:
                    print("Scheme for {} is {}".format(test_link,p_url[0]))
            
            #add the link to the list to be returned
            if test_link is not None and self.root in clean_url and urlerror == 0:
                return_list.append(urlopen(clean_url).geturl())

        return return_list
            
        
    def sort_list(self, l):
        return sorted(l)
    
    def test_url(self, url=None):
        if url is None:
            url = self.root
            
        try:
            read_text = urlopen(url).read()
            #add that link to the links that you've tested
            return read_text
        
        except URLError:
            print("{} is not a valid url.".format(url))
            return None
    
    def test_http(self, url=None):
        if url is None:
            url = self.root
        if isinstance(url,str):
            if url[0:4] != "http":
                return "http://" + url
            else:
                return url
        else:
            return None
    
    def drop_trail_slash(self,url,test_link):
        if len(url) > 0:
            print(url, test_link)
            if url[-1] == "/" and test_link[0] == "/":
                if len(test_link)==1:
                    return test_link
                else:
                    return test_link[1:]
            else:
                return test_link
            
        else:
            return test_link
        
    def drop_lead_dirnav(self, url):
        if len(url) > 1:
            if url[0:2] == "./":
                return url[1:]
            else:
                return url
        else:
            return url

In [10]:
class ACO():
    def __init__(self, aco_id, home_webpage, pr_webpage):
        self.config = pdfkit.configuration(wkhtmltopdf='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe')
        self.aco_id = aco_id
        self.lan_root = "\\\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM708_Dellva\Working\Compliance\Gov_Body_Review\GoverningBodyMaterial_ByACO\{}\\".format(self.aco_id)
        
        self.home_page = WebRoot(home_webpage, self.aco_id)
        self.home_page = self.home_page.test_http()
        
        self.pr_webpage = WebRoot(pr_webpage, self.aco_id)
        self.pr_webpage = self.pr_webpage.test_http()
        self.webpage_error = ""
        
    def __str__(self):
        return self.aco_id
    
    def __repr__(self):
        return self.aco_id
        
    def take_web_picture(self):
        options = {'quiet': ''}
        try:
            pdfkit.from_url(self.pr_webpage, self.lan_root + "public_reporting_webpage.pdf", configuration = self.config, options = options)
            self.webpage_error = "None"
            print("{} webpage downloaded.".format(self.aco_id))
        except OSError:
            self.webpage_error = "Yes"
            print("Error in downloading {}'s webpage.".format(self.aco_id))

    
    

In [11]:
downloads = r"\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\CM708_Dellva\Working\Compliance\Compliance_Data\Updated_HPMS_Downloads\\"
downloads_files = os.listdir(downloads)
for file in downloads_files:
    if "~$" not in file and "ManAgreement" in file:
        managree = pd.read_excel(downloads+file)
        managree = variable_standardize(managree)

managree.termination_date = pd.to_datetime(managree.termination_date, errors='coerce')
managree.current_start_date = pd.to_datetime(managree.current_start_date, errors='coerce')

#this grabs any ACO who has a termination date in the future
active_termed = managree[managree.termination_date.dt.date > datetime.date.today()]

#this grabs any ACO with no termination date
active = managree[managree.current_agreement_status == "Approved"]

#and this puts those two together to form the list of active ACOs as of today
managree = pd.concat([active_termed,active], ignore_index=True).sort_values('aco_id')

#get rid of ACOs starting in 2015 or 2018
managree = managree[(managree.current_start_date.dt.year != 2015) & (managree.current_start_date.dt.year != 2018)]

managree = managree[['aco_id','aco_legal_name','aco_web_page','aco_public_reporting_web_page','current_mssp_track','snf_3-day_rule_waiver_agreement_status']]

print("{} ACOs for picture day".format(managree.aco_id.count()))
managree.head()


371 ACOs for picture day


,aco_id,aco_legal_name,aco_web_page,aco_public_reporting_web_page,current_mssp_track,snf_3-day_rule_waiver_agreement_status
313,A1001,"Palm Beach Accountable Care Organization, LLC",www.pbaco.org,http://www.pbaco.org/PublicReporting,1,Withdrawn
384,A1002,Chinese Community Accountable Care Organizatio...,www.ccaco.org,www.ccaco.org/about/facts/,1,NaN
113,A1006,"Hackensack Physician-Hospital Alliance ACO, LLC",http://www.hackensackumc.org/patients-visitors...,http://www.hackensackumc.org/patients-visitors...,1,NaN
0,A1021,Concord Elliot ACO LLC,http://www.concordhospital.org/about/accountab...,http://www.concordhospital.org/about/accountab...,1,NaN
62,A1023,"Arizona Connected Care, LLC",www.azconnectedcare.org,http://www.azconnectedcare.org/who-we-are/publ...,1,NaN


In [13]:
aco_webpages = []
for i, row in managree.iterrows():
    print(row.aco_public_reporting_web_page)
    aco = ACO(row.aco_id, row.aco_web_page, row.aco_public_reporting_web_page)
    aco.take_web_picture()
    row.webpage_error = aco.webpage_error
    if i == 20:
        break

http://www.pbaco.org/PublicReporting
A1001 webpage downloaded.
www.ccaco.org/about/facts/
A1002 webpage downloaded.
http://www.hackensackumc.org/patients-visitors/billing-and-insurance/accountable-care-organization/public-reporting-information/
A1006 webpage downloaded.
http://www.concordhospital.org/about/accountable-care-organization/
A1021 webpage downloaded.
http://www.azconnectedcare.org/who-we-are/public-reporting-information/
Error in downloading A1023's webpage.
https://sitemanager.acsysinteractive.com/vSiteManager/AtlanticHealth/Public/Upload/Docs/A1026%20AHS%20ACO%20LLC%20%20Public%20Reporting%20%202018.pdf
Error in downloading A1026's webpage.
http://www.flphysicianstrust.org/index.php?id=aco
A1027 webpage downloaded.
http://www.premieraco.com/about-us.html
Error in downloading A1032's webpage.
http://www.advocatehealth.com/mssp
A1033 webpage downloaded.
http://www.rgvaco.com/public.html
A1037 webpage downloaded.
http://www.westfloridaaco.com/public-reporting/
A1041 webpage 

In [ ]:
url = "www.ccaco.org/about/"
read_text = urlopen(url).read()
soup = BeautifulSoup(read_text,"lxml")
for link in soup.find_all("a"):
    print(link.get('href'))

In [20]:
aco1 = WebRoot("ccaco.org")
aco1.extract()
print(aco1)

Iteration 0
Site Links are:
['ccaco.org']
Tested Links are:
[]
Extracting links for ccaco.org
<a href="javascript:imagezoom_close()" style="color:#606060;">close</a>
parsed url is: ParseResult(scheme='javascript', netloc='', path='imagezoom_close()', params='', query='', fragment='')
cleaned url is: javascript:imagezoom_close()
Scheme for javascript:imagezoom_close() is javascript
<a href="/"><img border="0" src="/_images_site/_common/title.png" width="400"/></a>
parsed url is: ParseResult(scheme='http', netloc='', path='/', params='', query='', fragment='')
cleaned url is: http:///


URLError: <urlopen error no host given>